# 神经网络模型

接下来以下面神经网络模型为例，说明神经网络中正向传播和反向传播过程及代码实现

<img src='./datasets/backpropagation3.png' height=50% width = 50%>

## 正向传播
(1)输入层神经元$i_1,i_2$,输入层到隐藏层处理过程

$$HiddenNeth_1 = w_1i_1+w_2i_2 + b_1$$
$$HiddenNeth_2 = w_3i_1+w_4i_2 + b_1$$
$$h_1 = sigmoid(HiddenNeth_1)$$
$$h_2 = sigmoid(HiddenNeth_2)$$

(2)隐藏层：神经元$h_1,h_2$,隐藏层到输出层处理过程

$$OutputNeto_1 = w_5h_1+w_6h_2$$
$$OutputNeto_2 = w_7h_1+w_8h_2$$
$$o_1 = sigmoid(OutputNeto_1)$$
$$o_2 = sigmoid(OutputNeto_2)$$

## 反向传播

反向传播是根据每次前向传播得到的误差来评估每个神经网络层中的权重对其的影响程度，再根据链式规则和梯度下降对权重进行更新的过程,下面以对更新权重$w_5和w_1$为例进行说明

- totalo1表示参考值0.01
- totalo2参考值0.99

### 更新权重w_5
$$\frac{\partial totalo1}{\partial w_5} = \frac{\partial totalo1}{\partial o_1} * \frac{\partial o_1}{\partial outputneto_1} * \frac{\partial outputneto_1}{\partial w_5}$$
其中：$$\frac{\partial totalo1}{\partial o_1} = \frac{\partial \frac{1}{2}{(totalo_1 - o_1)}^2}{\partial o_1} = -(totalo_1 -o_1)$$

$$\frac{\partial o_1}{\partial outputneto_1} = o_1(1-o_1)$$

$$\frac{\partial outputneto_1}{\partial w_5} = h_1$$

则：$$\frac{\partial totalo1}{\partial w_5} = -(totalo_1 -o_1)o_1(1-o_1)h_1$$
更新w_5的值：$$w_5new = w_5 - learningrate * \frac{\partial totalo1}{\partial w_5}$$

### 更新权重w_1

$求w_1的偏导与求w_5偏导唯一的区别在于，w_1的值影响了o_1和o_2$,所以
$$\frac{\partial total}{\partial w_1} = \frac{\partial totalol}{\partial w_1} + \frac{\partial totalo2}{\partial w_1}$$

接下来的求导思路和上面一样：
$$\frac{\partial totalol}{\partial w_1} = \frac{\partial totalol}{\partial h1} * \frac{\partial h1}{\partial hiddenneth1} * \frac{\partial hiddenneth1}{\partial w_1}$$

$$\frac{\partial total}{\partial w_1} = -(totalo1 - o1)o1(1-o1)w_5*h1(1-h1)i1 + -(totalo2 - o2)o2(1-o2)w_7*h1(1-h1)i1$$

# 代码实现正向和方向传播算法

In [3]:
import random
import math

class NeuralNetwork:
    learning_rate = 0.5
    def __init__(self,input_num,hidden_num,output_num,input_hidden_weights=None,
                 input_hidden_bias=None,hidden_output_weights=None,hidden_output_bias=None):
        
        self.input_num = input_num
        
        # 构建掩藏层
        self.hidden_layer = NeuralLayer(hidden_num,input_hidden_bias)
        # 构建输出层
        self.output_layer = NeuralLayer(output_num,hidden_output_bias)    
        # 初始化输入层到隐藏层权重
        self.init_input_to_hidden_weights(input_hidden_weights)
        # 初始化隐藏层到输出层权重
        self.init_hidden_to_output_weights(hidden_output_weights)
        
    def init_input_to_hidden_weights(self,weights):
        weight_num = 0
        for i_num in range(len(self.hidden_layer.neurons)):
            for o_num in range(self.input_num):
                if weights is None:
                    self.hidden_layer.neurons[i_num].weights.append(random.random())
                else:
                    self.hidden_layer.neurons[i_num].weights.append(weights[weight_num])
                weight_num += 1
                
    def init_hidden_to_output_weights(self,weights):
        weight_num = 0
        for i_num in range(len(self.output_layer.neurons)):
            for o_num in range(len(self.hidden_layer.neurons)):
                if weights is None:
                    self.output_layer.neurons[i_num].weights.append(random.random())
                else:
                    self.output_layer.neurons[i_num].weights.append(weights[weight_num])
                weight_num += 1
    
    def inspect(self):
        print('..................')
        print('input inspect:',[i for i in self.inputs])
        print('..................')
        print('hidden inspect:')
        self.hidden_layer.inspect()
        print('..................')
        print('output inspect:')
        self.output_layer.inspect()
        print('..................')
        
    def forward(self,inputs):
        hidden_layer_outout = self.hidden_layer.forward(inputs)
        print('hidden_layer_outout',hidden_layer_outout)
        ouput_layer_ouput = self.output_layer.forward(hidden_layer_outout)
        print('ouput_layer_ouput',ouput_layer_ouput)
        return ouput_layer_ouput
        
    def train(self,x,y):
        ouput_layer_ouput = self.forward(x)
        
        # 求total / neto的偏导
        total_o_pd = [0] * len(self.output_layer.neurons)
        for o in range(len(self.output_layer.neurons)):
            total_o_pd[o] = self.output_layer.neurons[o].calculate_total_net_pd(y[o]) 
            
        # 求total / h的偏导 = total.1 / h的偏导 + total.2 / h的偏导
        total_neth_pd = [0] * len(self.hidden_layer.neurons)
        for h in range(len(self.hidden_layer.neurons)):
            total_h_pd = 0
            for o in range(len(self.output_layer.neurons)):
                total_h_pd += total_o_pd[o] * self.hidden_layer.neurons[h].weights[o]
            total_neth_pd[h] = total_h_pd * self.output_layer.neurons[h].calculate_output_net_pd()
            
        # 更新输出层神经元权重
        for o in range(len(self.output_layer.neurons)):
            for ho_w in range(len(self.output_layer.neurons[o].weights)):
                ho_w_gradient = total_o_pd[o] * self.output_layer.neurons[o].calculate_net_linear_pd(ho_w)
                self.output_layer.neurons[o].weights[ho_w] -= self.learning_rate * ho_w_gradient
                
        # 更新隐藏层神经元权重
        for h in range(len(self.hidden_layer.neurons)):
            for ih_w in range(len(self.hidden_layer.neurons[h].weights)):
                ih_w_gradient = total_neth_pd[h] * self.hidden_layer.neurons[h].calculate_net_linear_pd(ih_w)
                self.hidden_layer.neurons[h].weights[ih_w] -= self.learning_rate * ih_w_gradient
                
    def calculate_total_error(self, training_sets):
        total_error = 0
        for t in range(len(training_sets)):
            training_inputs, training_outputs = training_sets[t]
            self.forward(training_inputs)
            for o in range(len(training_outputs)):
                total_error += self.output_layer.neurons[o].calculate_error(training_outputs[o])
        return total_error
    
class NeuralLayer:
    def __init__(self,neural_num,bias):
        self.bias = bias if bias else random.random()
        
        self.neurons = []
        
        for i in range(neural_num):
            self.neurons.append(Neuron(self.bias))
            
    def inspect(self):
        print('weights:',[neuron.weights for neuron in self.neurons])
        print('bias:',[neuron.bias for neuron in self.neurons])
      
    def get_output(self,inputs):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.output)
        return outputs
    
    def forward(self,inputs):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.calculate_output(inputs))
        return outputs
    
class Neuron:
    def __init__(self,bias):
        self.bias = bias        
        self.weights = []
        
    def calculate_output(self,inputs):
        self.inputs = inputs 
        total_net_outputs = self.calculate_total_net_output()
        self.output = self.sigmoid(total_net_outputs)
        return self.output
    
    def calculate_total_net_output(self):
        total = 0
        for i in range(len(self.inputs)):
            total += self.inputs[i] * self.weights[i]
        return total + self.bias
        
    def sigmoid(self,total_net_input):
        return 1 / (1 + math.exp(-total_net_input))
    
    def calculate_total_output_pd(self,total_output):
        return -(total_output - self.output)
    
    def calculate_output_net_pd(self):
        return self.output * (1 - self.output)
    
    def calculate_total_net_pd(self,total_output):
        return self.calculate_total_output_pd(total_output) * self.calculate_output_net_pd()
        
    def calculate_net_linear_pd(self,index):
        return self.inputs[index]
    
    def calculate_error(self, target_output):
        return 0.5 * (target_output - self.output) ** 2

In [9]:
nn = NeuralNetwork(2, 2, 2, input_hidden_weights=[0.15, 0.2, 0.25, 0.3], input_hidden_bias=0.35, 
                   hidden_output_weights=[0.4, 0.45, 0.5, 0.55], hidden_output_bias=0.6)
for i in range(10000):
    nn.train([0.05, 0.1], [0.01, 0.99])
    print(i, round(nn.calculate_total_error([[[0.05, 0.1], [0.01, 0.99]]]), 9))

hidden_layer_outout [0.5932699921071872, 0.596884378259767]
ouput_layer_ouput [0.7513650695523157, 0.7729284653214625]
hidden_layer_outout [0.5932662857458805, 0.5968782561589732]
ouput_layer_ouput [0.7420894573166411, 0.775286681791022]
0 0.291028391
hidden_layer_outout [0.5932662857458805, 0.5968782561589732]
ouput_layer_ouput [0.7420894573166411, 0.775286681791022]
hidden_layer_outout [0.5932623797502049, 0.5968720205376407]
ouput_layer_ouput [0.7324789213021788, 0.7775850216973027]
1 0.283547957
hidden_layer_outout [0.5932623797502049, 0.5968720205376407]
ouput_layer_ouput [0.7324789213021788, 0.7775850216973027]
hidden_layer_outout [0.5932582778661456, 0.5968656853189723]
ouput_layer_ouput [0.7225410209751278, 0.7798256906644981]
2 0.275943973
hidden_layer_outout [0.5932582778661456, 0.5968656853189723]
ouput_layer_ouput [0.7225410209751278, 0.7798256906644981]
hidden_layer_outout [0.5932539857457403, 0.5968592652405116]
ouput_layer_ouput [0.7122866732919141, 0.7820107943242337]
3

ouput_layer_ouput [0.12191085204138331, 0.9017605780393962]
172 0.010155117
hidden_layer_outout [0.5930290830920871, 0.5965963925934072]
ouput_layer_ouput [0.12191085204138331, 0.9017605780393962]
hidden_layer_outout [0.5930290430180494, 0.5965963036643693]
ouput_layer_ouput [0.12145787729755847, 0.9020052988115713]
173 0.010082963
hidden_layer_outout [0.5930290430180494, 0.5965963036643693]
ouput_layer_ouput [0.12145787729755847, 0.9020052988115713]
hidden_layer_outout [0.5930290039042683, 0.5965962162529418]
ouput_layer_ouput [0.12100961566608275, 0.9022482596881405]
174 0.010011751
hidden_layer_outout [0.5930290039042683, 0.5965962162529418]
ouput_layer_ouput [0.12100961566608275, 0.9022482596881405]
hidden_layer_outout [0.593028965728284, 0.59659613032824]
ouput_layer_ouput [0.12056598942659974, 0.9024894809864716]
175 0.009941464
hidden_layer_outout [0.593028965728284, 0.59659613032824]
ouput_layer_ouput [0.12056598942659974, 0.9024894809864716]
hidden_layer_outout [0.593028928468

hidden_layer_outout [0.5930277141138582, 0.5965914185825424]
ouput_layer_ouput [0.08577371472284627, 0.9240122879221594]
hidden_layer_outout [0.5930277167171217, 0.5965914067855901]
ouput_layer_ouput [0.0856090079947153, 0.9241273041818285]
306 0.005027967
hidden_layer_outout [0.5930277167171217, 0.5965914067855901]
ouput_layer_ouput [0.0856090079947153, 0.9241273041818285]
hidden_layer_outout [0.5930277193876708, 0.5965913951617163]
ouput_layer_ouput [0.08544523032484122, 0.9242418014049728]
307 0.005008062
hidden_layer_outout [0.5930277193876708, 0.5965913951617163]
ouput_layer_ouput [0.08544523032484122, 0.9242418014049728]
hidden_layer_outout [0.5930277221242741, 0.5965913837086184]
ouput_layer_ouput [0.08528237324915611, 0.9243557833910823]
308 0.004988299
hidden_layer_outout [0.5930277221242741, 0.5965913837086184]
ouput_layer_ouput [0.08528237324915611, 0.9243557833910823]
hidden_layer_outout [0.5930277249257221, 0.5965913724240288]
ouput_layer_ouput [0.08512042840933239, 0.9244

ouput_layer_ouput [0.06971421520145431, 0.9358511940999515]
hidden_layer_outout [0.5930283634443605, 0.596590786197491]
ouput_layer_ouput [0.06962540529572943, 0.935920206178839]
439 0.003239907
hidden_layer_outout [0.5930283634443605, 0.596590786197491]
ouput_layer_ouput [0.06962540529572943, 0.935920206178839]
hidden_layer_outout [0.5930283691899437, 0.5965907857104567]
ouput_layer_ouput [0.06953693623722311, 0.9359889923063183]
440 0.003230918
hidden_layer_outout [0.5930283691899437, 0.5965907857104567]
ouput_layer_ouput [0.06953693623722311, 0.9359889923063183]
hidden_layer_outout [0.5930283749370624, 0.5965907852580308]
ouput_layer_ouput [0.069448805898383, 0.936057553688157]
441 0.003221974
hidden_layer_outout [0.5930283749370624, 0.5965907852580308]
ouput_layer_ouput [0.069448805898383, 0.936057553688157]
hidden_layer_outout [0.5930283806855792, 0.5965907848398313]
ouput_layer_ouput [0.06936101216996421, 0.936125891521237]
442 0.003213075
hidden_layer_outout [0.5930283806855792,

hidden_layer_outout [0.5930291074033255, 0.5965909199179837]
ouput_layer_ouput [0.06019985606587295, 0.9434663836270388]
572 0.002342702
hidden_layer_outout [0.5930291074033255, 0.5965909199179837]
ouput_layer_ouput [0.06019985606587295, 0.9434663836270388]
hidden_layer_outout [0.5930291126861063, 0.5965909218605187]
ouput_layer_ouput [0.0601430307684665, 0.9435132035961071]
573 0.002337673
hidden_layer_outout [0.5930291126861063, 0.5965909218605187]
ouput_layer_ouput [0.0601430307684665, 0.9435132035961071]
hidden_layer_outout [0.593029117962971, 0.596590923810951]
ouput_layer_ouput [0.060086370001763666, 0.9435599036840872]
574 0.002332664
hidden_layer_outout [0.593029117962971, 0.596590923810951]
ouput_layer_ouput [0.060086370001763666, 0.9435599036840872]
hidden_layer_outout [0.5930291232339078, 0.5965909257691888]
ouput_layer_ouput [0.060029872988198854, 0.9436064843926265]
575 0.002327673
hidden_layer_outout [0.5930291232339078, 0.5965909257691888]
ouput_layer_ouput [0.0600298729

...

hidden_layer_outout [0.5930355362904329, 0.5965959772245898]
ouput_layer_ouput [0.0166266817292679, 0.9834175243492883]
hidden_layer_outout [0.5930355364088958, 0.5965959773375819]
ouput_layer_ouput [0.016626054949154753, 0.9834181436940597]
9568 4.3613e-05
hidden_layer_outout [0.5930355364088958, 0.5965959773375819]
ouput_layer_ouput [0.016626054949154753, 0.9834181436940597]
hidden_layer_outout [0.5930355365273392, 0.5965959774505555]
ouput_layer_ouput [0.016625428274775297, 0.9834187629350793]
9569 4.3604e-05
hidden_layer_outout [0.5930355365273392, 0.5965959774505555]
ouput_layer_ouput [0.016625428274775297, 0.9834187629350793]
hidden_layer_outout [0.593035536645763, 0.5965959775635108]
ouput_layer_ouput [0.016624801706101283, 0.9834193820723748]
9570 4.3596e-05
hidden_layer_outout [0.593035536645763, 0.5965959775635108]
ouput_layer_ouput [0.016624801706101283, 0.9834193820723748]
hidden_layer_outout [0.5930355367641672, 0.5965959776764477]
ouput_layer_ouput [0.016624175243104458, 

9667 4.2809e-05
hidden_layer_outout [0.5930355480407463, 0.596595988433569]
ouput_layer_ouput [0.016564522567449926, 0.9834789496704452]
hidden_layer_outout [0.5930355481572788, 0.5965959885447462]
ouput_layer_ouput [0.01656390622239182, 0.9834795587747819]
9668 4.2801e-05
hidden_layer_outout [0.5930355481572788, 0.5965959885447462]
ouput_layer_ouput [0.01656390622239182, 0.9834795587747819]
hidden_layer_outout [0.5930355482737921, 0.5965959886559054]
ouput_layer_ouput [0.016563289980290424, 0.983480167778072]
9669 4.2792e-05
hidden_layer_outout [0.5930355482737921, 0.5965959886559054]
ouput_layer_ouput [0.016563289980290424, 0.983480167778072]
hidden_layer_outout [0.5930355483902865, 0.5965959887670467]
ouput_layer_ouput [0.016562673841118526, 0.9834807766803421]
9670 4.2784e-05
hidden_layer_outout [0.5930355483902865, 0.5965959887670467]
ouput_layer_ouput [0.016562673841118526, 0.9834807766803421]
hidden_layer_outout [0.5930355485067618, 0.5965959888781701]
ouput_layer_ouput [0.01656

9767 4.2017e-05
hidden_layer_outout [0.5930355596004727, 0.596595999463336]
ouput_layer_ouput [0.016503393329409692, 0.9835393641799834]
hidden_layer_outout [0.5930355597151241, 0.5965959995727439]
ouput_layer_ouput [0.01650278714671877, 0.9835399633095606]
9768 4.2009e-05
hidden_layer_outout [0.5930355597151241, 0.5965959995727439]
ouput_layer_ouput [0.01650278714671877, 0.9835399633095606]
hidden_layer_outout [0.5930355598297569, 0.5965959996821344]
ouput_layer_ouput [0.01650218106430802, 0.9835405623406993]
9769 4.2001e-05
hidden_layer_outout [0.5930355598297569, 0.5965959996821344]
ouput_layer_ouput [0.01650218106430802, 0.9835405623406993]
hidden_layer_outout [0.5930355599443711, 0.5965959997915073]
ouput_layer_ouput [0.016501575082151165, 0.9835411612734253]
9770 4.1994e-05
hidden_layer_outout [0.5930355599443711, 0.5965959997915073]
ouput_layer_ouput [0.016501575082151165, 0.9835411612734253]
hidden_layer_outout [0.5930355600589666, 0.5965959999008629]
ouput_layer_ouput [0.01650

9867 4.1246e-05
hidden_layer_outout [0.5930355709744578, 0.5965960103183758]
ouput_layer_ouput [0.016443267235805973, 0.9835987939720332]
hidden_layer_outout [0.5930355710872757, 0.5965960104260586]
ouput_layer_ouput [0.016442670952604543, 0.9835993833830371]
9868 4.1238e-05
hidden_layer_outout [0.5930355710872757, 0.5965960104260586]
ouput_layer_ouput [0.016442670952604543, 0.9835993833830371]
hidden_layer_outout [0.5930355712000753, 0.5965960105337244]
ouput_layer_ouput [0.016442074767102518, 0.9835999726981179]
9869 4.123e-05
hidden_layer_outout [0.5930355712000753, 0.5965960105337244]
ouput_layer_ouput [0.016442074767102518, 0.9835999726981179]
hidden_layer_outout [0.5930355713128569, 0.5965960106413732]
ouput_layer_ouput [0.01644147867927455, 0.9836005619173005]
9870 4.1223e-05
hidden_layer_outout [0.5930355713128569, 0.5965960106413732]
ouput_layer_ouput [0.01644147867927455, 0.9836005619173005]
hidden_layer_outout [0.5930355714256206, 0.5965960107490048]
ouput_layer_ouput [0.016

9967 4.0494e-05
hidden_layer_outout [0.5930355821673773, 0.5965960210030196]
ouput_layer_ouput [0.01638411846591902, 0.9836572642132898]
hidden_layer_outout [0.5930355822784075, 0.5965960211090197]
ouput_layer_ouput [0.016383531828691487, 0.9836578441528473]
9968 4.0486e-05
hidden_layer_outout [0.5930355822784075, 0.5965960211090197]
ouput_layer_ouput [0.016383531828691487, 0.9836578441528473]
hidden_layer_outout [0.5930355823894201, 0.5965960212150032]
ouput_layer_ouput [0.01638294528667401, 0.9836584239989086]
9969 4.0479e-05
hidden_layer_outout [0.5930355823894201, 0.5965960212150032]
ouput_layer_ouput [0.01638294528667401, 0.9836584239989086]
hidden_layer_outout [0.593035582500415, 0.5965960213209702]
ouput_layer_ouput [0.01638235883984215, 0.9836590037514978]
9970 4.0471e-05
hidden_layer_outout [0.593035582500415, 0.5965960213209702]
ouput_layer_ouput [0.01638235883984215, 0.9836590037514978]
hidden_layer_outout [0.5930355826113923, 0.5965960214269206]
ouput_layer_ouput [0.0163817

迭代10000次后结果：

hidden_layer_outout [0.5930355856011269, 0.5965960242813179]

ouput_layer_ouput [0.016365976874939202, 0.9836751989707726]

hidden_layer_outout [0.5930355857116121, 0.5965960243868048]

ouput_layer_ouput [0.016365393177895763, 0.9836757760229975]

9999 4.0257e-05